### Introduction

In this project, we'll be looking at the questions that the TV show Jeopardy typically asks to try and discern some patterns that could offer us a competitive edge on the show.

We'll be working with a dataset called jeopardy.csv, containing 20,000 rows of Jeopardy questions and related information. We can see the data and columns below.

In [56]:
import pandas as pd

data = pd.read_csv("jeopardy.csv")
data.columns = ["Show Number", "Air Date", "Round", "Category", "Value", "Question", "Answer"]
data.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


### Cleaning the Data

Before we analyze the data it's a good idea to normalize the data in some of the columns. We'll remove capitalization and punctuation in the Question, and Answer columns.

In [57]:
import re

def remove_grammar(string):
    string = string.lower()
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    return string

data["Question"] = data["Question"].apply(remove_grammar)
data["Answer"] = data["Answer"].apply(remove_grammar)

We also want to clean up the Value column and convert it to an integer. Not all values in the column are numerical, so we'll have to replace non-numerical strings with an integer manually. Fortunately, this is only the case for "None" values so we can just replace non-numerical strings with 0.

In [58]:
def value_to_integer(string):
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    try:
        string = int(string)
    except:
        string = 0
    return string

data["Value"] = data["Value"].apply(value_to_integer)

Finally, let's convert the values in the Air Date column to datetime objects.

In [59]:
data["Air Date"] = pd.to_datetime(data["Air Date"])

### Devising a Jeopardy Strategy

To devise a strategy for Jeopary, we want to begin by thinking about what kinds of things we want to study. If the same questions appear on Jeopardy often, then it would be good to start by studying the questions that have appeared on past episodes of Jeopardy. Or, if the answer can be inferred from the question, we may not need to study at all!

Let's start by looking at the latter of these possibilities. While there are lots of kinds of clues questions can give you that allow you to infer the answer, we'll only be looking at question and answer pairs in which the question contains part of, or the, answer.

We'll do this by comparing the list of words in each question with the list of words in that question's answer. For each pair, we'll count how many matches there are, and then see what proportio of the answer is a match by dividing the number of words in the answer by the number of matches. We'll do this for every question and answer pair, adding the results to a new column.

In [60]:
def answer_matches_question(row):
    question = row["Question"]
    answer = row["Answer"]
    split_question = question.split()
    split_answer = answer.split()
    split_answer = [x for x in split_answer if x != "the"]
    match_count = 0
    if len(split_answer) == 0:
        return(0)
    else:
        for word in split_answer:
            if word in split_question:
                match_count += 1
    proportion_matches = match_count / len(split_answer)
    return(proportion_matches)

In [61]:
data["answer_in_question"] = data.apply(answer_matches_question, axis = 1)
mean_answer_in_question = data["answer_in_question"].mean()
mean_answer_in_question

0.058347444789267004

It looks like the strategy of not studying at all would not go well! Only 5.8% of the words in the answers match words in the question. Given that most answers are one or two words, that's a pretty low hit rate.

Let's now look at how often questions are repeats of old ones, to see whether it's worth studying former Jeopardy questions. Rather than look for exact matches of questions, we'll look for questions which contain similar terms to previous ones.

In [68]:
sorted_dates = data["Air Date"].sort_values(ascending = True)

terms_used = set()
# We're creating a set here rather than a list as sets are immutable and contain only unique items.
# This will prevent us adding duplicate words to the set which we then have to iterate through, thus speeding up our loop.
question_overlap = []

for i, row in data.iterrows():
    words = row["Question"].split()
    words = [x for x in words if len(x) > 5]
    # To save time, we'll try to filter out words which are likely to be less signifcant to the meaning of the question.
    match_count = 0
    for word in words:
        if word in terms_used:
            match_count += 1
    for word in words:
        terms_used.add(word)
    if len(words) > 0:
        match_count /= len(words)
    question_overlap.append(match_count)

data["question_overlap"] = question_overlap
mean_overlap = data["question_overlap"].mean()
mean_overlap

0.6908737315671878

70% of the content of new questions (counting only words with 6 or more letters) has been used in previous questions. At first glance this seems like quite a high proportion, but there are a few things that we need to note about the figure.

The first is that questions which appear earlier in our dataset are less likely to match previous questions as there are fewer of them. The first question we iterate through in our above code will have no words that match the previous question. This means that, as time goes on, it's more likely that questions will be repeats. This further means that the mean is not a very good figure for determining the likelihood of a repeat question for future shows, as the mean is affected by the early shows where the chance of a repeat question was significantly lower. This factor leads to an underestimate of the mean.

Secondly, the same words may appear in questions that occur in the same show, but even if some of the words are the same, we can be quite certain (unless the makers of the show have made a significant error) that the semantic content of that question is distinct. Currently, however, our code doesn't account for this, and so this factor leads to an overestimate of the mean.

Thirdly, words used in previous question can appear in current questions, but the semantic content of the question can be distinct. For example, words like defeated or winner can refer to a wide variety of categories and have totally different content. This is the major issue with the figure we have above. How many of the words that are repeated actually indicate that the question with the same semantic content is being asked? My guess is that the answer is relatively low, and that most of the matches come from words which are frequently used in questions of a particular category, like city, country, etc. This factor leads to an overestimate of the mean with regards to the information we'd hoped it might give us.

Fourthly, our threshold for meaningful words is hard to evaluate. We miss out words like for, in, the, of, etc., which would greatly influence the mean and so we rightly wish to disregard. However, we also miss out words like Italy, game, and so on which indicate categories which the question refers to and which are relevant if repeated.

All this being said, 70% is still quite high, and even if our mean is a drastic overestimate, studying old Jeopardy questions may well be a good strategy.

### Analyzing Question Categories

A better measure than something so complex might be to just do a simple count of unique words in questions and then manually inspect the results to see which categories come up frequently. Let's do this now.

We'll reduce the threshold for significant words to those words with more than 3 letters. We'll still have words like which and from, but will remove words like for, of, the, in at hopefully less information cost. The best way to filter for articles, prepositions, etc. would be to create a dictionary with such words and then remove them from the questions, but we won't be doing that here.

In [78]:
word_counts = {}

for i, row in data.iterrows():
    words = row["Question"].split()
    words = [x for x in words if len(x) > 3]
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

for w in sorted(word_counts, key = word_counts.get, reverse = True):
    print(w, word_counts[w])

this 11676
from 1851
with 1658
these 1389
that 1348
name 1025
first 949
city 581
when 577
about 549
called 521
named 513
like 489
have 483
country 476
were 450
seen 446
state 443
type 440
after 426
clue 420
film 414
made 401
used 386
title 370
your 368
crew 368
also 365
which 361
known 352
here 327
into 326
word 324
years 316
only 311
said 297
played 297
wrote 287
became 287
term 275
over 267
before 267
novel 265
world 261
president 258
capital 257
american 257
part 251
than 249
famous 246
targetblankherea 244
french 243
last 242
some 241
book 236
their 235
king 234
been 231
more 230
most 227
show 224
home 220
time 218
island 216
john 212
they 210
song 210
means 209
where 209
great 208
found 205
what 198
born 193
just 191
play 190
river 188
make 186
people 184
national 183
many 182
work 180
largest 179
little 178
life 176
group 173
there 171
around 169
comes 169
once 169
star 168
back 168
whose 166
british 166
house 165
south 165
author 164
meaning 162
during 161
while 161
dont 161
gre

certain 28
sure 28
code 28
legendary 28
important 28
targetblankhi 28
titles 28
rose 28
organ 28
equal 28
department 28
lower 28
remember 28
production 28
founder 28
ford 28
hold 28
control 28
signed 28
medical 28
knight 28
coin 28
hours 27
glass 27
launched 27
basketball 27
4letter 27
foot 27
tony 27
youd 27
pennsylvania 27
becomes 27
womens 27
raised 27
spot 27
cars 27
murder 27
test 27
wore 27
outside 27
received 27
speed 27
bush 27
write 27
ruler 27
christian 27
1957 27
instead 27
appropriately 27
mile 27
holy 27
structure 27
1951 27
jane 27
temple 27
pirate 27
cape 27
evil 27
action 27
return 27
lewis 27
francisco 27
least 27
theory 26
snow 26
1920s 26
divided 26
bring 26
francis 26
discovery 26
skin 26
machine 26
student 26
salt 26
coined 26
course 26
flowers 26
refer 26
tail 26
annual 26
1948 26
count 26
bell 26
cluea 26
soldiers 26
artists 26
titled 26
kill 26
slang 26
dress 26
cold 26
walk 26
devoted 26
wanted 26
owner 26
cathedral 26
ring 26
pope 26
deal 26
succeeded 26
orlea

publication 11
experiments 11
peanut 11
attorney 11
democrat 11
celebration 11
muslim 11
wales 11
whiskey 11
wave 11
separated 11
potable 11
stamp 11
telescope 11
causes 11
underground 11
entered 11
bass 11
boxer 11
monarch 11
englands 11
failed 11
onto 11
marilyn 11
3word 11
fever 11
crane 11
drop 11
executive 11
maria 11
priest 11
adapted 11
talking 11
lucy 11
wagon 11
1927 11
problem 11
increase 11
cooked 11
apply 11
bacon 11
hanks 11
signs 11
estimated 11
dame 11
sight 11
pull 11
receive 11
fresh 11
exercise 11
hong 11
mistress 11
happens 11
camera 11
dear 11
burned 11
faith 11
1903 11
carter 11
interest 11
sheep 11
sells 11
finish 11
arent 11
direct 11
pulled 11
prior 11
tribute 11
teenage 11
keeps 11
lisa 11
2letter 11
jose 11
zealand 11
eaten 11
shortly 11
ernest 11
thick 11
fact 11
1857 11
presented 11
crystal 11
assassination 11
baked 11
bergman 11
ordered 11
quiet 11
vessel 11
cardinal 11
myth 11
potent 11
greeks 11
commercial 11
swing 11
beast 11
booth 11
idol 11
jungle 11
s

cafe 8
dressed 8
fate 8
birthstone 8
rump 8
fords 8
suicide 8
defensive 8
ulysses 8
charlemagne 8
remake 8
virginias 8
beware 8
passion 8
briefly 8
duncan 8
avenue 8
merged 8
leonard 8
fastener 8
phil 8
problems 8
1850 8
jets 8
alive 8
ideas 8
dwarf 8
flew 8
martini 8
chancellor 8
pigs 8
canary 8
nixon 8
nest 8
anatomy 8
filmed 8
puerto 8
merry 8
enemy 8
sudden 8
boulevard 8
kenya 8
gothic 8
verne 8
bunny 8
brands 8
expensive 8
gorge 8
pluto 8
joan 8
feed 8
corner 8
designated 8
flies 8
1492 8
readers 8
closed 8
nick 8
ambassador 8
hate 8
meets 8
rachel 8
equator 8
dash 8
walking 8
particle 8
falling 8
wonderful 8
frogs 8
sunset 8
rolled 8
eisenhower 8
bloody 8
edmund 8
delicacy 8
sally 8
visitors 8
cowboys 8
aromatic 8
hood 8
oman 8
herb 8
essential 8
wilkes 8
cotton 8
chaplin 8
classical 8
stroke 8
1804 8
establish 8
eden 8
strikes 8
hugo 8
naught 8
fatal 8
spacecraft 8
bilbao 8
galileo 7
sunshine 7
basically 7
headlines 7
agreed 7
creating 7
seats 7
buddy 7
pointed 7
organizations 7

obtained 6
slain 6
1874 6
jamaica 6
suburb 6
tunnel 6
rage 6
isaac 6
lovely 6
annually 6
secrets 6
animation 6
goods 6
cubs 6
essay 6
brief 6
1832 6
dining 6
hybrid 6
thrilled 6
inspiration 6
twist 6
denmarks 6
100000 6
slowly 6
targetblanksofia 6
spaina 6
lara 6
lethal 6
billions 6
bust 6
familys 6
listen 6
delight 6
spielberg 6
wilde 6
oldfashioned 6
maple 6
waltz 6
sudan 6
1818 6
maintains 6
monty 6
uniform 6
heroic 6
paints 6
buying 6
quoted 6
trust 6
tasty 6
storage 6
prominent 6
librarian 6
nassau 6
canterbury 6
retreat 6
busiest 6
contained 6
buddhist 6
offensive 6
100th 6
assume 6
worms 6
virtue 6
scholars 6
available 6
scoring 6
websters 6
driver 6
300000 6
conflict 6
invaders 6
carthage 6
tools 6
playboy 6
failure 6
joining 6
cajun 6
abstract 6
priests 6
hitchcock 6
louise 6
dominican 6
pursue 6
jackie 6
universitya 6
extra 6
messenger 6
accessory 6
hyde 6
practical 6
melville 6
slightly 6
leaf 6
aside 6
demonstrated 6
ivan 6
danced 6
succession 6
fixed 6
warned 6
monaco 6
ac

...

ypsilanti 1
eisenach 1
reformer 1
wartburg 1
hrefhttpwwwjarchivecommedia20071106j23wmvherea 1
hacksaw 1
bechamel 1
materialism 1
bobo 1
1729 1
binghi 1
hrefhttpwwwjarchivecommedia20071106dj06jpgalex 1
picchua 1
hrefhttpwwwjarchivecommedia20071106dj06ajpg 1
targetblankthat 1
ledgea 1
aligns 1
mcentires 1
18month 1
1667 1
gravitation 1
oreos 1
janeites 1
hrefhttpwwwjarchivecommedia20071106dj07jpgalex 1
hrefhttpwwwjarchivecommedia20071106dj07ajpg 1
targetblankhiram 1
binghama 1
affectionately 1
undiscovered 1
ornaments 1
kidds 1
rails 1
hrefhttpwwwjarchivecommedia20071106dj08jpgalex 1
hrefhttpwwwjarchivecommedia20071106dj08ajpg 1
lovett 1
tuba 1
refute 1
uncertainty 1
hrefhttpwwwjarchivecommedia20071106dj23jpg 1
zoya 1
hrefhttpwwwjarchivecommedia20071106dj09jpgalex 1
sedona 1
hrefhttpwwwjarchivecommedia20071106dj09ajpg 1
mckechnie 1
cassie 1
damboise 1
1628 1
alzheimers 1
disapprovingly 1
wagonmaker 1
hrefhttpwwwjarchivecommedia20071106dj10jpgalex 1
talons 1
hrefhttpwwwjarchivecommedia200

bloomberg 1
activitiesone 1
awaytheres 1
encore 1
foisthe 1
manoa 1
arranges 1
embargo 1
mentors 1
mariology 1
cousine 1
crooning 1
hrefhttpwwwjarchivecommedia20080509j24jpg 1
sather 1
californiaberkeleya 1
hrefhttpwwwjarchivecommedia20080509j24ajpg 1
targetblankfiat 1
luxa 1
pushes 1
boomers 1
daccord 1
hairdos 1
metaphorically 1
lending 1
parlezvous 1
allemand 1
hrefhttpwwwjarchivecommedia20080509dj17jpg 1
hrefhttpwwwjarchivecommedia20080509dj17ajpg 1
pressed 1
nittany 1
lovin 1
litter 1
faraway 1
evenly 1
stadiums 1
buckeyes 1
hrefhttpwwwjarchivecommedia20080509dj02jpg 1
hrefhttpwwwjarchivecommedia20080509dj28jpg 1
hrefhttpwwwjarchivecommedia20080509dj19jpg 1
shorten 1
hrefhttpwwwjarchivecommedia20080509dj19ajpg 1
fern 1
warts 1
corkscrew 1
reworked 1
kroc 1
hrefhttpwwwjarchivecommedia20080509dj14jpg 1
hrefhttpwwwjarchivecommedia20080509dj30jpg 1
hrefhttpwwwjarchivecommedia20080509dj21jpg 1
illustrate 1
hrefhttpwwwjarchivecommedia20080509dj21ajpg 1
newtonian 1
marcy 1
vestments 1
pl

droplets 1
bakery 1
creamy 1
cakelike 1
unicorn 1
drey 1
teaberry 1
shrivers 1
hrefhttpwwwjarchivecommedia20110712dj25jpg 1
hrefhttpwwwjarchivecommedia20110712dj25ajpg 1
byre 1
booklet 1
punches 1
bootlegger 1
dampier 1
inhabitant 1
221b 1
hrefhttpwwwjarchivecommedia20091117j30wmvalex 1
4972a 1
muppeteer 1
clash 1
giggle 1
brangus 1
valid 1
melvin 1
dummars 1
googlyeyed 1
gobbling 1
hermes 1
starving 1
heartland 1
scramble 1
squabble 1
prostitute 1
serum 1
accidentwhoa 1
spleen 1
budds 1
ethelreds 1
hrefhttpwwwjarchivecommedia20091117j12jpg 1
targetblankthereas 1
intestate 1
sweatinducing 1
bath 1
hrefhttpwwwjarchivecommedia20091117dj01jpg 1
sauteed 1
cambrai 1
meck 1
preferred 1
symbolic 1
hrefhttpwwwjarchivecommedia20091117dj20jpg 1
outgunned 1
sovietmade 1
ludovico 1
sforza 1
grandest 1
oncenomadic 1
hrefhttpwwwjarchivecommedia20091117dj03jpg 1
lawrys 1
misnomer 1
keni 1
keto 1
shinhoto 1
chiha 1
kessler 1
hrefhttpwwwjarchivecommedia20091117dj27jpg 1
alvar 1
aalto 1
tartan 1
marinad

qaywayn 1
iskenderun 1
islahiye 1
fessenden 1
abbreviations 1
omitted 1
equally 1
offenders 1
clayand 1
occupational 1
pastorelli 1
cracker 1
eldin 1
hrefhttpwwwjarchivecommedia19991217j21jpg 1
alexakis 1
evercleara 1
alienated 1
semicolon 1
doldrums 1
yarmulke 1
mid80s 1
geddes 1
ellie 1
glidin 1
onehorse 1
paragraphs 1
compost 1
penzance 1
morel 1
gorshin 1
pirating 1
trumpets 1
cursus 1
hedgehog 1
hemiechinus 1
sensory 1
toasted 1
eureka 1
hrefhttpwwwjarchivecommedia20060725j01mp3thisa 1
nasal 1
hrefhttpwwwjarchivecommedia20060725j02mp3thisa 1
offically 1
cheneys 1
flecked 1
nationsnot 1
irelandsfirst 1
1142 1
hrefhttpwwwjarchivecommedia20060725j03mp3herea 1
offenbachs 1
genevieve 1
hrefhttpwwwjarchivecommedia20060725j18jpg 1
setter 1
hrefhttpwwwjarchivecommedia20060725j18ajpg 1
targetblankstylea 1
mozambique 1
decolletage 1
fortuitously 1
hrefhttpwwwjarchivecommedia20060725j05mp3herea 1
deacon 1
2masted 1
rigged 1
hrefhttpwwwjarchivecommedia20060725j06mp3followinga 1
proven 1
terps

ulcer 1
ranitidine 1
hydrochloride 1
smedley 1
peel 1
hrefhttpwwwjarchivecommedia20040707dj16jpg 1
targetblankmerlina 1
victorians 1
invincible 1
planner 1
immunosuppressant 1
transplants 1
fellowsoldiers 1
ivanhoe 1
uncover 1
2340mile 1
talleyrand 1
intriguing 1
sawatch 1
epperson 1
lemonade 1
hatchetwielding 1
smasher 1
hatchetface 1
ortega 1
kisses 1
113mile 1
pesticides 1
oleancontaining 1
unmake 1
licentious 1
pence 1
westphall 1
ehrlich 1
pfennigs 1
unwanted 1
meanie 1
spaghettini 1
mcdermott 1
1670s 1
ruccolo 1
traylor 1
kopecks 1
firewood 1
bind 1
gala 1
berkshires 1
darned 1
urich 1
centessimi 1
squaredoff 1
eartoear 1
biblicalera 1
lovd 1
wisely 1
participants 1
1504 1
hrefhttpwwwjarchivecommedia20060127j20jpg 1
replicas 1
trophies 1
hrefhttpwwwjarchivecommedia20060127j20ajpg 1
targetblank97 1
mastersa 1
hrefhttpwwwjarchivecommedia20060127j20bjpg 1
targetblank2000 1
opena 1
manley 1
abercrombie 1
zambia 1
noyes 1
excommunicated 1
hrefhttpwwwjarchivecommedia20060127j21jpg 1
hr

There are a little under 20000 questions in the dataset, so there don't appear to be any categories that come up with great regularity. However, we can discern some more common categories from the above, namely: city, country, state, film, crew, played, president, american, french, king, song, and so on. These represent categories like history, geography, music, and sports/games.

Each of those words appear in at least 1% of questions on the show. That means it's unlikely for any one of them to appear on the show, but they're still better bets for categorical study than some others.

### Maximizing Question Value

Let's opt for a different strategy now. We want to get the most value out of our studying hours, so let's direct them towards categories or questions that have a high monetary value.

We can get a rough sense of this by assigning a monetary value to individual words that appeared in our terms_used set previously. We'll begin by classifying each question as either high or low value. Next, we'll define a function which will calculate the value of each word by counting how many times each word appears in high and low value questions respectively. We'll then apply this function to a small selection of words, giving us two frequencies for each word. Finally, we'll compare these frequencies with the expected frequencies of each word with a chi-squared test. The results of this test will indicate whether there is a correlation between the word and the value of the question.

In [90]:
def find_worth(row):
    value = row["Value"]
    if value > 800:
        worth = 1
    else:
        worth = 0
    return (worth)

data["high_value"] = data.apply(find_worth, axis = 1)

In [97]:
def word_worth(word):
    low_count = 0
    high_count = 0
    for i, row in data.iterrows():
        if word in row["Question"].split():
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return(high_count, low_count)

terms = list(terms_used)[:5]
# To make computation quicker, we'll only take the first 5 terms from terms_used.
observed_counts = []
for term in terms:
    observed_counts.append(word_worth(term))

observed_counts

[(0, 2), (0, 1), (1, 1), (0, 1), (0, 1)]

We have the number of high and low value counts for each of the five words we've selected. We now need to find the expected value counts so that we can work out whether any of these results are significant.

In [103]:
from scipy.stats import chisquare

high_value_count = data[data["high_value"] == 1].shape[0]
low_value_count = data[data["high_value"] == 0].shape[0]

chi_squared = []

for observed in observed_counts:
    total = observed[0] + observed[1] # Number of times each word appears in the dataset
    total_proportion = total / data.shape[0] # Proportion of times each word appears in dataset
    high_value_expected = total_proportion * high_value_count
    low_value_expected = total_proportion * low_value_count
    
    observed = np.array([observed[0], observed[1]])
    expected = np.array([high_value_expected, low_value_expected])
    
    chi_squared.append(chisquare(observed, expected))

chi_squared
# "statistic" below is the chi-squared value.

[Power_divergenceResult(statistic=0.803925692253768, pvalue=0.3699222378079571),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.4448774816612795, pvalue=0.5047776487545996),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469)]

All of the p-values for these words are very high, meaning the probability that the given words would appear in high/low value questions at the frequencies they did by random chance is high. That is, if the words in high value questions and low value questions were decided randomly, there is a good chance that the frequencies we observed would occur as a result of that random process. As such, it is unlikely that there is a correlation between the words we've investigated and the value of the question. For reference, a p-value is considered statistically significant if it is less than 0.05 (or 5%).

Similarly, the chi-squared values are all low, meaning there is little difference between the observed and expected values, again indicating a lack of a correlation between our selected words and the value of the question.

Now, it should be noted that chi-squared tests become less useful as the sample size decreases. Each word that we've selected only appears twice, meaning that our sample size is extremely small. This makes chi-squared an almost useless measure in this situation. As expected, we're unable to conclude anything much from this line of investigation.

### Conclusion

To conclude, then, our best strategy appears to be either studying past Jeopardy questions, or studying specific categories which the Jeopardy questions tend to fall into.